In [198]:
import pandas as pd
import numpy as np
import folium
from folium import plugins
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.style.use('ggplot')
%matplotlib inline

import seaborn as sns
sns.set(style="whitegrid")

from geopy.geocoders import Nominatim

In [199]:
data = pd.read_csv('input.csv')
data['change'] = data['worthChange'].apply(lambda v: 'gain' if v > 0 else 'loss')

In [200]:
## TODO
# Top 10 Richest in a visualization.
# Gain losses in one single visualization.

In [201]:
data.head()

,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,country,timestamp,realTimeWorth,realTimeRank,change
0,Bill Gates,Gates,bill-gates,bill-gates,54.866,60.0,Microsoft,Technology,M,United States,1.470000e+12,79507.055,1,gain
1,Amancio Ortega,Ortega,amancio-ortega,amancio-ortega,1330.609,80.0,Zara,Fashion and Retail,M,Spain,1.470000e+12,77489.082,2,gain
2,Jeff Bezos,Bezos,jeff-bezos,jeff-bezos,-276.709,52.0,Amazon.com,Technology,M,United States,1.470000e+12,66922.566,3,loss
3,Warren Buffett,Buffett,warren-buffett,warren-buffett,-114.906,86.0,Berkshire Hathaway,Finance and Investments,M,United States,1.470000e+12,65306.558,4,loss
4,Mark Zuckerberg,Zuckerberg,mark-zuckerberg,mark-zuckerberg,-176.700,32.0,Facebook,Technology,M,United States,1.470000e+12,55227.791,5,loss


In [202]:
"""
Industry - Gains
Industry - Losses
Average Billions per Country
Countries with 1 billionaire
"""

'\nIndustry - Gains\nIndustry - Losses\nAverage Billions per Country\nCountries with 1 billionaire\n'

In [203]:
from bokeh.charts import show, Area
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [204]:
# Top 10 Countries

data['country_count'] = 1
countries_b = data.groupby('country').count().reset_index().sort_values(by='country_count', ascending=False)
country_count = countries_b[:10]


# Top Countries for Female Entrepreneurs
countries_female = data[data['gender'] == 'F']
countries_female_c = countries_female.groupby('country').count().reset_index().sort_values(by='country_count', ascending=False)[:10]

# Top Industries
data = data.drop('country_count', axis=1)
data['industry_count'] = 1
industry_count_b = data.groupby('industry').count().reset_index().sort_values(by='industry_count', ascending=False)[:10]
print(industry_count_b.head())

                   industry  name  lastName  uri  imageUri  worthChange  age  \
5   Finance and Investments   277       277  277       277          275  272   
4        Fashion and Retail   232       232  232       232          228  224   
16               Technology   172       172  172       172          166  168   
13              Real Estate   172       172  172       172          168  166   
10            Manufacturing   170       170  170       170          160  166   

    source  gender  country  timestamp  realTimeWorth  realTimeRank  change  \
5      277     276      277        277            277           277     277   
4      232     226      232        232            232           232     232   
16     172     171      172        172            172           172     172   
13     172     172      172        172            172           172     172   
10     170     170      170        170            170           170     170   

    industry_count  
5              277  
4 

In [205]:
country_count


,country,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,timestamp,realTimeWorth,realTimeRank,change,country_count
64,United States,555,555,555,555,540,551,555,555,550,555,555,555,555,555
9,China,269,269,269,269,254,264,269,269,269,269,269,269,269,269
18,Germany,120,120,120,120,120,111,120,120,118,120,120,120,120,120
23,India,97,97,97,97,94,92,97,97,97,97,97,97,97,97
48,Russia,83,83,83,83,83,83,83,83,83,83,83,83,83,83
21,Hong Kong,65,65,65,65,64,61,65,65,65,65,65,65,65,65
63,United Kingdom,53,53,53,53,52,50,53,53,51,53,53,53,53,53
6,Brazil,45,45,45,45,44,44,45,45,45,45,45,45,45,45
16,France,44,44,44,44,44,43,44,44,44,44,44,44,44,44
27,Italy,43,43,43,43,42,38,43,43,43,43,43,43,43,43


## Top 10 Countries

In [206]:
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr
from bokeh.charts import defaults

defaults.width = 500
defaults.height = 450

bar = Bar(country_count,
          values='country_count',
          label=CatAttr(columns=['country'], sort=False),
          legend='top_right',
          xlabel='Countries',
          ylabel='Count of Billionaires',
          title='Top 10 Countries',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [207]:
countries_gain_loss = data.groupby(['country','change']).size().reset_index().rename(columns={0:'count'})
countries_gain_loss = countries_gain_loss[countries_gain_loss['country'].isin(top_10_countries)]
top_10_countries = ['United States','China','Germany','India','Russia','Hong Kong','United Kingdom','Brazil','France','Italy']
countries_gain_loss['country'] = pd.Categorical(countries_gain_loss['country'], top_10_countries)
countries_gain_loss['change'] = pd.Categorical(countries_gain_loss['change'], ['gain','loss'])
countries_gain_loss = countries_gain_loss.sort_values(by=['country','change'])
print(countries_gain_loss.sort_values(by='country'))

defaults.width = 500
defaults.height = 450

bar = Bar(countries_gain_loss, 
          values='count', 
          label=CatAttr(columns=['country'], sort=False),
          stack='change',
          xlabel='Countries',
          ylabel='Count of Billionaires - Gain vs. Loss',
          title="Gain vs. Loss in Top 10 Countries", 
          legend='top_right', 
          agg ='sum',
          bar_width=0.6,
          tools="crosshair,save",
          color=["green", "red"])
show(bar)

            country change  count
109   United States   gain    140
110   United States   loss    415
14            China   gain    169
15            China   loss    100
31          Germany   gain     28
32          Germany   loss     92
39            India   gain     50
40            India   loss     47
81           Russia   loss     53
80           Russia   gain     30
36        Hong Kong   gain     40
37        Hong Kong   loss     25
107  United Kingdom   gain     15
108  United Kingdom   loss     38
8            Brazil   gain     22
9            Brazil   loss     23
28           France   gain     18
29           France   loss     26
47            Italy   gain     24
48            Italy   loss     19


## Top 10 Countries - Female Billionaires

In [208]:
defaults.width = 500
defaults.height = 450

bar = Bar(countries_female_c,
          values='country_count',
          label=CatAttr(columns=['country'], sort=False),
          legend='top_right',
          xlabel='Countries',
          ylabel='Count of Billionaires',
          title='Top Countries with Female Billionaires',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [209]:
countries_female_c

,country,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,timestamp,realTimeWorth,realTimeRank,change,country_count
32,United States,66,66,66,66,63,66,66,66,66,66,66,66,66,66
11,Germany,27,27,27,27,27,25,27,27,27,27,27,27,27,27
6,China,15,15,15,15,15,15,15,15,15,15,15,15,15,15
12,Hong Kong,10,10,10,10,10,8,10,10,10,10,10,10,10,10
3,Brazil,9,9,9,9,9,9,9,9,9,9,9,9,9,9
28,Switzerland,8,8,8,8,8,8,8,8,8,8,8,8,8,8
10,France,7,7,7,7,7,7,7,7,7,7,7,7,7,7
13,India,7,7,7,7,5,7,7,7,7,7,7,7,7,7
15,Italy,7,7,7,7,7,5,7,7,7,7,7,7,7,7
30,Turkey,5,5,5,5,5,5,5,5,5,5,5,5,5,5


## Top 10 Industries

In [210]:
from bokeh.charts import Bar
from bokeh.charts.attributes import CatAttr
from bokeh.charts import defaults

defaults.width = 550
defaults.height = 800

bar = Bar(industry_count_b,#.sort(columns='country_no', ascending=False)[:10], 
          values='industry_count',
          label=CatAttr(columns=['industry'], sort=False),
          legend='top_right',
          xlabel='Industries',
          ylabel='Count of Billionaires',
          title='Top 10 Industries',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [230]:
industry_count_b

,industry,name,lastName,uri,imageUri,worthChange,age,source,gender,country,timestamp,realTimeWorth,realTimeRank,change,industry_count
5,Finance and Investments,277,277,277,277,275,272,277,276,277,277,277,277,277,277
4,Fashion and Retail,232,232,232,232,228,224,232,226,232,232,232,232,232,232
16,Technology,172,172,172,172,166,168,172,171,172,172,172,172,172,172
13,Real Estate,172,172,172,172,168,166,172,172,172,172,172,172,172,172
10,Manufacturing,170,170,170,170,160,166,170,170,170,170,170,170,170,170
2,Diversified,155,155,155,155,151,151,155,155,155,155,155,155,155,155
6,Food and Beverage,147,147,147,147,142,139,147,144,147,147,147,147,147,147
8,Healthcare,125,125,125,125,121,121,125,125,125,125,125,125,125,125
3,Energy,90,90,90,90,88,89,90,90,90,90,90,90,90,90
11,Media & Entertainment,74,74,74,74,73,72,74,74,74,74,74,74,74,74


In [211]:
data['age'].max()

def age_bin(val):
    if not pd.isnull(val):
        val = int(val)
        if val in list(range(1,6)):
            bin_pos = 5
        elif val in list(range(6,11)):
            bin_pos = 10
        elif val in list(range(11,16)):
            bin_pos = 15
        elif val in list(range(16,21)):
            bin_pos = 20
        elif val in list(range(21,26)):
            bin_pos = 25  
        elif val in list(range(26,31)):
            bin_pos = 30
        elif val in list(range(31,36)):
            bin_pos = 35
        elif val in list(range(36,41)):
            bin_pos = 40
        elif val in list(range(41,46)):
            bin_pos = 45
        elif val in list(range(46,51)):
            bin_pos = 50
        elif val in list(range(51,56)):
            bin_pos = 55
        elif val in list(range(56,61)):
            bin_pos = 60
        elif val in list(range(61,66)):
            bin_pos = 65
        elif val in list(range(66,71)):
            bin_pos = 70
        elif val in list(range(71,76)):
            bin_pos = 75
        elif val in list(range(76,81)):
            bin_pos = 80
        elif val in list(range(81,86)):
            bin_pos = 85
        elif val in list(range(86,91)):
            bin_pos = 90
        elif val in list(range(91,96)):
            bin_pos = 95
        elif val in list(range(96,101)):
            bin_pos = 100
        elif val in list(range(101,106)):
            bin_pos = 105
        return bin_pos
    else:
        return np.nan


In [212]:
data.head()
data['billionaire_count'] = 1
age_dist = data.copy()
age_dist['age_bin'] = data['age'].apply(age_bin)
age_bin_count = age_dist[['age_bin','billionaire_count']].groupby('age_bin').sum().reset_index()
age_bin_count

,age_bin,billionaire_count
0,20.0,1
1,25.0,2
2,30.0,6
3,35.0,27
4,40.0,36
5,45.0,73
6,50.0,157
7,55.0,244
8,60.0,231
9,65.0,265


In [229]:
defaults.width = 600
defaults.height = 450

area = Area(age_bin_count, x='age_bin', y='billionaire_count', title="Billionares per Age", color="royalblue",
            xlabel="Miles Per Gallon", ylabel="Horsepower", tools="crosshair,save",)


show(area)

In [231]:
data[data['age'] > 100]

,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,country,timestamp,realTimeWorth,realTimeRank,change,industry_count,billionaire_count
614,"David Rockefeller, Sr.",Rockefeller,david-rockefeller-sr,david-rockefeller-sr,0.0,101.0,"oil, banking",Energy,M,United States,1.470000e+12,3150.0,614,loss,1,1


In [214]:
worth_change_data = data.groupby(['change']).size().reset_index().rename(columns={0:'count'})

defaults.width = 300
defaults.height = 450

bar = Bar(worth_change_data,
          values='count',
          label=CatAttr(columns=['change'], sort=False),
          legend='top_left',
          xlabel='Gain & Loss',
          ylabel='Count of Billionaires',
          title='Total Worth Change - Gain and Loss',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [215]:
worth_change_data

,change,count
0,gain,780
1,loss,1139


In [216]:
countries = list(data['country'].value_counts().index)

In [217]:
geolocator = Nominatim()

In [218]:
latitudes = []
longitudes = []

for country in countries:
    location = geolocator.geocode(country)
    latitudes.append(location.latitude)
    longitudes.append(location.longitude)

In [219]:
countries_data = pd.DataFrame({'countries': countries, 
                               'latitudes': latitudes, 
                               'longitudes': longitudes})

In [220]:
countries_b.head()

def get_countries_count(val):
    c_c_dict = dict(zip(countries_b['country'],countries_b['country_count']))
    return c_c_dict.get(val)
    

countries_data['billionaire_count'] = countries_data['countries'].apply(get_countries_count)

In [221]:
countries_map = folium.Map(location=[countries_data['latitudes'].mean(), countries_data['longitudes'].mean()], zoom_start=-10)
marker_cluster = folium.MarkerCluster().add_to(countries_map)
for name, row in countries_data.iterrows():
    folium.Marker([row["latitudes"], row["longitudes"]], popup="{0}: {1}".format(row["countries"], row["billionaire_count"])).add_to(marker_cluster)
countries_map.save('countries.html')

In [222]:
countries_map

## Digging Deeper into United States(count-555)

In [223]:
us_data = data[data['country'] == 'United States']
us_data.head()

,name,lastName,uri,imageUri,worthChange,age,source,industry,gender,country,timestamp,realTimeWorth,realTimeRank,change,industry_count,billionaire_count
0,Bill Gates,Gates,bill-gates,bill-gates,54.866,60.0,Microsoft,Technology,M,United States,1.470000e+12,79507.055,1,gain,1,1
2,Jeff Bezos,Bezos,jeff-bezos,jeff-bezos,-276.709,52.0,Amazon.com,Technology,M,United States,1.470000e+12,66922.566,3,loss,1,1
3,Warren Buffett,Buffett,warren-buffett,warren-buffett,-114.906,86.0,Berkshire Hathaway,Finance and Investments,M,United States,1.470000e+12,65306.558,4,loss,1,1
4,Mark Zuckerberg,Zuckerberg,mark-zuckerberg,mark-zuckerberg,-176.700,32.0,Facebook,Technology,M,United States,1.470000e+12,55227.791,5,loss,1,1
5,Larry Ellison,Ellison,larry-ellison,larry-ellison,335.942,72.0,Oracle,Technology,M,United States,1.470000e+12,49645.365,6,gain,1,1


In [224]:
us_gender = us_data.groupby(['gender']).size().reset_index().rename(columns={0:'count'})

defaults.width = 300
defaults.height = 450

bar = Bar(us_gender,
          values='count',
          label=CatAttr(columns=['gender'], sort=False),
          legend='top_left',
          xlabel='Gender',
          ylabel='Count of Billionaires',
          title='Male:Female Distribution in USA',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [225]:
# Top Industries in United States
us_industries = us_data.groupby(['industry']).size().reset_index().rename(columns={0:'count'}).sort_values(by='count',ascending=False)

defaults.width = 700
defaults.height = 650

bar = Bar(us_industries,
#           orientation="vertical",
          values='count',
          label=CatAttr(columns=['industry'], sort=False),
          legend='top_right',
          xlabel='Industry',
          ylabel='Count of Billionaires',
          title='Top Industries in USA',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [226]:
fin_inv_data = data[data['industry'] == 'Finance and Investments']
worth_change_data = fin_inv_data.groupby(['change']).size().reset_index().rename(columns={0:'count'})

defaults.width = 400
defaults.height = 450

bar = Bar(worth_change_data,
          values='count',
          label=CatAttr(columns=['change'], sort=False),
          legend='top_left',
          xlabel='Gain & Loss',
          ylabel='Count of Billionaires',
          title='Total Worth Change-Gain and Loss in Finance & Investments',
          color="royalblue",
          tools="crosshair,save",
          bar_width=0.6
         )
show(bar)

In [227]:
industry_gain_loss = data.groupby(['industry','change']).size().reset_index().rename(columns={0:'count'})
top_10_industries = ['Finance and Investments','Fashion and Retail','Real Estate','Technology','Manufacturing','Diversified','Food and Beverage','Healthcare','Energy','Media & Entertainment','Service','Construction & Engineering','Metals & Mining','Automotive','Logistics','Telecom','Gambling, Casinos','Sports']
industry_gain_loss['industry'] = pd.Categorical(industry_gain_loss['industry'], top_10_industries)
industry_gain_loss['change'] = pd.Categorical(industry_gain_loss['change'], ['gain','loss'])
industry_gain_loss = industry_gain_loss.sort_values(by=['industry','change'])


defaults.width = 500
defaults.height = 450

bar = Bar(industry_gain_loss, 
          values='count', 
#           label='industry', 
          label=CatAttr(columns=['industry'], sort=False),
          stack='change',
          xlabel='Countries',
          ylabel='Count of Billionaires - Gain vs. Loss',
          title="Gain vs. Loss in Top 10 Countries", 
          legend='top_right', 
#           agg ='sum',
          bar_width=0.6,
          tools="crosshair,save",
          color=["green", "red"])
show(bar)

In [228]:
industry_gain_loss.groupby('industry').sum().reset_index().sort_values(by='count', ascending=False)

,industry,count
0,Finance and Investments,277
1,Fashion and Retail,232
2,Real Estate,172
3,Technology,172
4,Manufacturing,170
5,Diversified,155
6,Food and Beverage,147
7,Healthcare,125
8,Energy,90
9,Media & Entertainment,74
